In [2]:
import os
import random
from typing import List
import twscrape

In [11]:
# set up directories

keyword_eng = "zainichi"
year = "2022"
os.mkdir(keyword_eng + "_" + str(year))

# single digit month
for i in range(1,10):
    path = keyword_eng + "_" + str(year) + "/" + str(year) + "-0" + str(i)
    os.mkdir(path) 

# double digit month
for j in range(10,13):
    path = keyword_eng + "_" + str(year) + "/" + str(year) + "-" + str(j)
    os.mkdir(path) 

In [5]:
# remove directories if needed
# !rm -rf 2008/

In [9]:
# collect tweets for 10 randomly sampled days of month at randomly sampled time of day
def scrape_tweets(year: str, months: List[str], days_in_month: int, keyword: str):
    range_days = list(range(1,days_in_month+1))
    range_times = list(range(0,24))

    # sample 10 random days and times of day for each month
    for m in months: 
        month_path = year + "/" + year + "-" + m
        days = sorted(random.sample(range_days, k=10))   # random days of month (no replacement)
        times = random.choices(range_times, k=10)   # random hours of day (replacement)
        
        # scrape tweets for the 10 picked days and times
        for t in range(10):     
            day = "0" + str(days[t]) if days[t] < 10 else str(days[t])
            time = "0" + str(times[t]) if times[t] < 10 else str(times[t])
            day_path = month_path + "/" + year + "-" + m + "-" + day + ".txt"
            command = 'twscrape search "' + keyword + ' since:' + year + '-' + m + '-' + day + '_' + time + ':00:00_UTC lang:ja"　> ' + day_path + ' --limit=4500'
            # os.system(command)
            print(command)

In [12]:
# for a full year
keyword = "在日コリアン"
year = "2022"

# 28 day months
scrape_tweets(year,["02"], 28, keyword)    # omit leap year 29th days for simplicity...?

# 30 day months
months_30 = ["04","06","09","11"]
scrape_tweets(year,months_30, 30, keyword)

# 31 day months
months_31 = ["01","03","05","07","08","10","12"]
scrape_tweets(year,months_31, 31, keyword)

sh: 2022/2022-02/2022-02-07.txt: No such file or directory
sh: 2022/2022-02/2022-02-10.txt: No such file or directory
sh: 2022/2022-02/2022-02-14.txt: No such file or directory
sh: 2022/2022-02/2022-02-15.txt: No such file or directory
sh: 2022/2022-02/2022-02-21.txt: No such file or directory
sh: 2022/2022-02/2022-02-23.txt: No such file or directory
sh: 2022/2022-02/2022-02-24.txt: No such file or directory
sh: 2022/2022-02/2022-02-25.txt: No such file or directory
sh: 2022/2022-02/2022-02-26.txt: No such file or directory
sh: 2022/2022-02/2022-02-27.txt: No such file or directory


TypeError: scrape_tweets() missing 1 required positional argument: 'keyword'

In [17]:
# concatenate .txt files into one file per month
year = "2022"
for root, dirs, files in os.walk("./" + year):
    for name in dirs:
      month_path = os.path.join(root, name)
      os.system("cat " + month_path + "/*.txt > " + name + ".txt")

In [22]:
# concatenate month .txt files into one file for the year
os.system("cat " + year + "-*.txt > " + keyword + "_" + year + "-all.txt")

0